In [1]:
import numpy as np
import pandas as pd
from hmmlearn.hmm import GaussianHMM
from sklearn.preprocessing import StandardScaler
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'numpy'

In [ ]:
class HMMTrader:
    def __init__(self, n_states=4, n_lookback=5):
        display(Markdown("### Init HMMTrader..."))
        """
        Initialize HMM Trading model
        
        Parameters:
        n_states (int): Number of hidden states in HMM
        n_lookback (int): Number of days to look back for features
        """
        self.n_states = n_states
        self.n_lookback = n_lookback
        self.hmm = GaussianHMM(
            n_components=n_states,
            covariance_type="full",
            n_iter=1000,
            random_state=42
        )
        self.scaler = StandardScaler()
        
    def _create_features(self, data):
        display(Markdown("### Create features"))
        """
        Create features from price data
        
        Parameters:
        data (pd.DataFrame): DataFrame with 'Close' prices
        
        Returns:
        np.array: Feature matrix
        """
        # Calculate returns
        returns = np.log(data['Close'] / data['Close'].shift(1)).dropna()
        
        # Calculate volatility
        volatility = returns.rolling(window=self.n_lookback).std().dropna()
        
        # Calculate moving averages
        ma5 = data['Close'].rolling(window=5).mean()
        ma20 = data['Close'].rolling(window=20).mean()
        
        # Calculate moving average crossover
        ma_crossover = (ma5 - ma20) / ma20
        
        # Create features matrix
        features = pd.concat([
            returns,
            volatility,
            ma_crossover.shift(1)
        ], axis=1).dropna()
        
        return self.scaler.fit_transform(features)
    
    def train(self, data):
        """
        Train the HMM model
        
        Parameters:
        data (pd.DataFrame): DataFrame with at least 'Close' price column
        """
        features = self._create_features(data)
        self.hmm.fit(features)
        
    def predict_states(self, data):
        """
        Predict hidden states for given data
        
        Parameters:
        data (pd.DataFrame): DataFrame with at least 'Close' price column
        
        Returns:
        np.array: Predicted states
        """
        features = self._create_features(data)
        return self.hmm.predict(features)
    
    def get_trading_signals(self, data):
        """
        Generate trading signals based on predicted states
        
        Parameters:
        data (pd.DataFrame): DataFrame with at least 'Close' price column
        
        Returns:
        pd.DataFrame: DataFrame with predictions and signals
        """
        features = self._create_features(data)
        states = self.hmm.predict(features)
        
        # Calculate state means for returns
        state_returns = {}
        returns = np.log(data['Close'] / data['Close'].shift(1)).dropna()
        
        for state in range(self.n_states):
            state_returns[state] = returns[states == state].mean()
        
        # Generate signals based on state expected returns
        signals = pd.Series(index=returns.index, data=0)
        
        for state in range(self.n_states):
            if state_returns[state] > 0:
                signals[states == state] = 1
            elif state_returns[state] < 0:
                signals[states == state] = -1
        
        return pd.DataFrame({
            'Returns': returns,
            'State': states,
            'Signal': signals
        })
    
    def backtest(self, data, initial_capital=100000):
        """
        Backtest the trading strategy
        
        Parameters:
        data (pd.DataFrame): DataFrame with at least 'Close' price column
        initial_capital (float): Initial capital for backtesting
        
        Returns:
        pd.DataFrame: DataFrame with backtest results
        """
        signals = self.get_trading_signals(data)
        
        # Calculate strategy returns
        strategy_returns = signals['Signal'].shift(1) * signals['Returns']
        
        # Calculate cumulative returns
        cumulative_returns = (1 + strategy_returns).cumprod()
        
        # Calculate portfolio value
        portfolio_value = initial_capital * cumulative_returns
        
        # Calculate metrics
        total_return = (portfolio_value.iloc[-1] - initial_capital) / initial_capital
        sharpe_ratio = np.sqrt(252) * strategy_returns.mean() / strategy_returns.std()
        max_drawdown = (portfolio_value / portfolio_value.cummax() - 1).min()
        
        results = pd.DataFrame({
            'Portfolio_Value': portfolio_value,
            'Strategy_Returns': strategy_returns,
            'Cumulative_Returns': cumulative_returns
        })
        
        print(f"Backtest Results:")
        print(f"Total Return: {total_return:.2%}")
        print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
        print(f"Max Drawdown: {max_drawdown:.2%}")
        
        return results

In [ ]:
# Example usage
display(Markdown("### Load def of Load&PrepareData"))
def load_and_prepare_data(file_path):
    """
    Load and prepare data for HMM trading
    
    Parameters:
    file_path (str): Path to CSV file with OHLCV data
    
    Returns:
    pd.DataFrame: Prepared DataFrame
    """
    df = pd.read_csv(file_path)
    
    #Conver
    
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')  # for seconds
    df.set_index('Timestamp', inplace=True)
    return df

In [ ]:
display(Markdown("### Starting the main script"))
# Load your data
# Example: df = load_and_prepare_data('AAPL_daily.csv')
df = load_and_prepare_data('/kaggle/input/bitcoin-historical-data/btcusd_1-min_data.csv')


In [ ]:
# Initialize model
hmm_trader = HMMTrader(n_states=4, n_lookback=5)

In [ ]:
# Split data into train and test
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]

# Train model
hmm_trader.train(train_data)

# Backtest on test data
results = hmm_trader.backtest(test_data)

In [ ]:
# Optional: Plot results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(results['Portfolio_Value'])
plt.title('Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.grid(True)
plt.show()